In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (roc_curve, auc, roc_auc_score,
                             confusion_matrix)
import matplotlib.pyplot as plt
from sklearn import preprocessing
import itertools
import pickle
from sklearn.utils import resample
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours, RandomUnderSampler, ClusterCentroids
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('given_dataset.csv')
df.drop('id', axis = 1, inplace = True)
df.dropna()
df.drop_duplicates()
df.head()

,A,B,C,D,E,F,G,T
0,8.341749e+07,2,124,4,0.144868,3.091977e-01,22,1
1,1.379494e+08,2,5,34,0.117274,9.492332e-03,0,1
2,1.293132e+08,2,324,18,0.006898,6.352157e-07,2,1
3,1.028544e+08,2,202,4,0.000000,8.980000e-02,0,1
4,2.784674e+09,2,281,56,0.000000,3.288574e-04,0,1


In [3]:
df['T'].value_counts()

1    2012677
0      29010
Name: T, dtype: int64

In [ ]:
# ones = df[df['T'] == 1].sample(n = 50000).index
# zeros = df[df['T'] == 0].index

# df = df.loc[ones.union(zeros)]
# smote = SMOTE(sampling_strategy=1.0, k_neighbors=2)
# X_train, y_train = smote.fit_resample(X_train, y_train)

In [24]:
X = df[['A', 'B', 'C', 'D', 'E', 'F', 'G']]
y = df['T']

# RUS
# X_maj = df[df['T'] == 1]
# X_min = df[df['T'] == 0]
# X_maj_rus = resample(X_maj, replace = False, n_samples = 2*len(X_min), random_state = 111)
# X_rus = pd.concat([X_maj_rus, X_min])
# X_train = X_rus.drop(['T'], axis = 1)
# y_train = X_rus['T']
# Cluster Centroids
# cc = ClusterCentroids()
# X, y = cc.fit_sample(X, y)
# SMOTE

# over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy = 0.4)
X, y = under.fit_resample(X, y)

smote = SMOTE(sampling_strategy=0.7, k_neighbors=7)
X_train, y_train = smote.fit_resample(X, y)
# steps = [('o', over), ('u', under)]
# pipeline = Pipeline(steps=steps)
# X, y = pipeline.fit_resample(X, y)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42)

In [25]:
y_train.value_counts()

1    72525
0    50767
Name: T, dtype: int64

In [29]:
# cv_method = RepeatedStratifiedKFold(n_splits=10,n_repeats=1,random_state=42)
# model_RFC = RandomForestClassifier(n_estimators = 100, max_depth = 9)
# model_RFC.fit(X_train, y_train)

# adb = AdaBoostClassifier(RandomForestClassifier(n_estimators = 100, max_depth = 9),n_estimators=100,learning_rate=0.5)
param = {'max_depth': 10, 'eta': 1, 'objective': 'binary:logistic', 'scale_pos_weight' : 0.7}
model = XGBClassifier(**param)
# model = GradientBoostingClassifier(max_depth = 10, scale_pos_weight = 0.5)
metLearn=CalibratedClassifierCV(model, method='isotonic', cv=2)
metLearn.fit(X_train, y_train)
# model.fit(X, y)

# param_grid = { 
#     'n_estimators': [200, 500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4,5,6,7,8],
#     'criterion' :['gini', 'entropy']
# }
# gs_RFC = GridSearchCV(estimator=model_RFC, 
#                       param_grid=param_grid, 
#                       cv=cv_method,
#                       verbose=1, 
#                       scoring='roc_auc')

CalibratedClassifierCV(base_estimator=XGBClassifier(base_score=None,
                                                    booster=None,
                                                    colsample_bylevel=None,
                                                    colsample_bynode=None,
                                                    colsample_bytree=None,
                                                    eta=1, gamma=None,
                                                    gpu_id=None,
                                                    importance_type='gain',
                                                    interaction_constraints=None,
                                                    learning_rate=None,
                                                    max_delta_step=None,
                                                    max_depth=10,
                                                    min_child_weight=None,
                                                    missing

In [ ]:
gs_RFC.fit(X_train, y_train)

In [ ]:
gs_RFC.best_params_

In [30]:
df_test = pd.read_csv('to_predict.csv')
df_test.drop_duplicates()
df_test.dropna()
op = df_test[['id']]
df_test.drop('id', axis = 1, inplace = True)
df_test.head()

,A,B,C,D,E,F,G
0,3.048551e+08,2,126,40,0.000000,0.673500,0
1,8.306319e+07,2,272,18,0.006898,0.342363,0
2,1.773707e+09,2,121,2,0.000000,0.673500,0
3,1.000000e+08,1,121,0,0.000000,0.673500,0
4,3.300000e+07,1,71,4,0.000000,0.168375,0


In [31]:
X = df_test[['A', 'B', 'C', 'D', 'E', 'F', 'G']]
ans = metLearn.predict(X)
op.insert(1, 'T', ans, True)
op.head()

,id,T
0,2041687,1
1,2041688,0
2,2041689,1
3,2041690,1
4,2041691,0


In [32]:
op.to_csv('Hope5.csv', index = False)

In [ ]:
df.shape

In [ ]:
X.shape

In [ ]:
y.shape